## 1. Favorite Genre For Each User

In [17]:
import sqlite3
import pandas as pd


In [ ]:
# ratings_df = pd.read_sql_query("SELECT * FROM Movies_ratings", conn)
# genres_df = pd.read_sql_query("SELECT * FROM Movie_Genres", conn)
ratings_df=pd.read_csv("data/Movies_ratings_df.csv")
genres_df=pd.read_csv("data/Movie_Genres_df.csv")

merged = pd.merge(ratings_df, genres_df, on="movie_id")

In [19]:
genre_cols = [col for col in genres_df.columns if col != "movie_id"]

for genre in genre_cols:
    merged[genre + "_score"] = merged[genre] * merged["rating"]


In [20]:
genre_score_cols = [g + "_score" for g in genre_cols]

genre_counts = merged.groupby("user_id")[genre_cols].sum()

genre_totals = merged.groupby("user_id")[genre_score_cols].sum()

user_genre_scores = genre_totals.values / (genre_counts.values + 20)
user_genre_scores = pd.DataFrame(user_genre_scores, columns=genre_score_cols, index=genre_totals.index)
user_genre_scores = user_genre_scores.fillna(0)
print(user_genre_scores)

         Action_score  Adventure_score  Animation_score  Biography_score  \
user_id                                                                    
0            3.351955         3.300546         2.760274              0.0   
1            3.252174         3.400000         3.174419              0.0   
2            2.539474         2.485981         1.608108              0.0   
3            3.309211         3.452778         3.226316              0.0   
4            2.539216         2.660156         2.755952              0.0   
...               ...              ...              ...              ...   
864          3.150538         3.322727         2.446809              0.0   
865          3.436813         3.409756         3.054348              0.0   
866          3.670520         3.529762         2.685484              0.0   
867          2.415842         2.922819         3.028409              0.0   
868          2.718310         2.922222         2.771084              0.0   

         Co

In [21]:
favorite_genre = user_genre_scores.idxmax(axis=1).str.replace("_score", "")
favorite_genre.name = "favorite_genre"

user_fav_df = pd.DataFrame(favorite_genre)

user_fav_df['user_id'] = user_fav_df.index

print(user_fav_df.head())


        favorite_genre  user_id
user_id                        
0                Drama        0
1                Drama        1
2                Drama        2
3                Drama        3
4                Drama        4


In [ ]:
all_genres = [
    "Action", "Adventure", "Animation", "Biography", "Comedy", "Crime", "Documentary", "Drama", "Family",
    "Fantasy", "History", "Horror", "Music", "Mystery", "Romance", "Science_Fiction", "Thriller", "War", "Western"
]

genre_dict = {genre: idx + 1 for idx, genre in enumerate(all_genres)}
user_fav_df['genre_id'] = user_fav_df['favorite_genre'].map(genre_dict)
user_fav_df = user_fav_df[['user_id', 'genre_id']]
print(user_fav_df.head())
user_fav_df.drop(columns=["user_id"])
user_fav_df.to_csv('data/User_Favorite_Genre.csv')



         user_id  genre_id
user_id                   
0              0         8
1              1         8
2              2         8
3              3         8
4              4         8


## 2. Ratings STD For Each Movie

In [ ]:
# ratings_df = pd.read_sql_query("SELECT movie_id, rating FROM Movies_ratings", conn)
ratings_df=pd.read_csv("data/Movies_ratings_df.csv")
rating_std_df = ratings_df.groupby("movie_id")["rating"].std().reset_index()
rating_std_df.columns = ["movie_id", "rating_std"]

rating_std_df.to_csv("data/Movies_Ratings_Variance.csv", index=False)


## 3. Analyzing Each Person's Personality

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# ratings_df = pd.read_sql_query("SELECT * FROM Movies_ratings", conn)
# genres_df = pd.read_sql_query("SELECT * FROM Movie_Genres", conn)
# movies_df = pd.read_sql_query("SELECT * FROM Final_Movie_Data", conn)
ratings_df=pd.read_csv("data/Movies_ratings_df.csv")
genres_df=pd.read_csv("data/Movie_Genres_df.csv")
movies_df=pd.read_csv("data/Final_Movie_Data_df.csv")
user_rating_var = ratings_df.groupby('user_id')['rating'].var().reset_index()
user_rating_var.columns = ['user_id', 'rating_variance']

high_rated = ratings_df[ratings_df['rating'] > 3.5]
high_rated_genres = pd.merge(high_rated, genres_df, on='movie_id', how='left')

genre_cols = [col for col in genres_df.columns if col != 'movie_id']
user_genre_counts = high_rated_genres.groupby('user_id')[genre_cols].sum()

user_genre_diversity = user_genre_counts.var(axis=1).reset_index()
user_genre_diversity.columns = ['user_id', 'genre_diversity']

long_movies = movies_df[movies_df['duration'] > 120][['movie_id']]
long_high_rated = pd.merge(high_rated, long_movies, on='movie_id', how='inner')
user_attention_span = long_high_rated.groupby('user_id').size().reset_index(name='long_high_rated_count')

def classify_big_five_traits(user_genre_scores, threshold=3):
    trait_genres = {
        "Openness": ["Fantasy_score", "Science_Fiction_score", "Adventure_score", "Animation_score", "Music_score"],
        "Conscientiousness": ["History_score", "War_score"],
        "Extraversion": ["Action_score", "Comedy_score", "Adventure_score"],
        "Agreeableness": ["Family_score", "Romance_score", "Animation_score"],
        "Neuroticism": ["Thriller_score", "Horror_score", "Mystery_score", "Crime_score"]
    }

    trait_df = pd.DataFrame(index=user_genre_scores.index)

    for trait, genres in trait_genres.items():
        relevant_scores = user_genre_scores[genres]
        trait_df[trait] = relevant_scores.gt(threshold).all(axis=1).astype(int)
        trait_df[trait] = trait_df[trait].fillna(0)

    return trait_df

merged_df = pd.merge(user_rating_var, user_genre_diversity, on='user_id', how='outer')
merged_df = pd.merge(merged_df, user_attention_span, on='user_id', how='outer').fillna(0)

big_five_df = classify_big_five_traits(user_genre_scores)

final_features = pd.concat([merged_df['user_id'], big_five_df, merged_df[['rating_variance', 'genre_diversity', 'long_high_rated_count']]], axis=1)


scaler = MinMaxScaler()

final_features[['long_high_rated_count', 'genre_diversity']] = scaler.fit_transform(
    final_features[['long_high_rated_count', 'genre_diversity']]
)
final_features.to_csv("data/user_psychological_features.csv", index=False)
final_features.head()

,user_id,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism,rating_variance,genre_diversity,long_high_rated_count
0,0.0,0.0,0.0,1.0,0.0,0.0,0.320030,0.009637,0.094632
1,1.0,0.0,0.0,1.0,1.0,0.0,0.736749,0.007372,0.071942
2,2.0,0.0,0.0,0.0,0.0,0.0,1.067529,0.002106,0.043719
3,3.0,0.0,0.0,1.0,1.0,0.0,1.092089,0.013128,0.110127
4,4.0,0.0,0.0,0.0,0.0,0.0,0.912184,0.002556,0.040952
